# Building a database of MLB statistics
For this project, we'll build a database of baseball statistics dating back to the 1800s starting with four csv files from Retrosheet. We'll: 
- read in and do some preliminary exploration of the csvs to get a handle on what we're working with
- import the data into SQLite
- design a normalized database schema
- create tables for our schema
- insert the data into our schema
Let's get started!

In [1]:
# import libraries
import pandas as pd
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)
import sqlite3
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# helper functions
def run_query(q):
    # opens a connection to database to run a query, q
    # returns a pandas dataframe
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql(q, conn)

In [3]:
def run_command(q):
    # opens a connection to database to run a command with no output
    with sqlite3.connect('mlb.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(q)

In [4]:
def show_tables():
    # returns a list of all tables and views in our database
    # triple quotes allows us to spread it across several lines for readability
    q = """
            SELECT 
                name, 
                type 
            FROM sqlite_master 
            WHERE type IN (\"table\",\"view\");
        """
    return run_query(q)

In [5]:
# uncomment and run the line below to delete db for a rebuild
#!rm mlb.db

In [6]:
# read in helper text file describing columns of main document
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


In [7]:
# import game_log.csv
game_log = pd.read_csv("game_log.csv", low_memory=False)
game_log.shape

(171907, 161)

In [8]:
game_log.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54.0,D,NaN,NaN,NaN,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,6.0,1.0,NaN,-1.0,NaN,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,NaN,-1.0,NaN,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,18,54.0,D,NaN,NaN,NaN,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,NaN,5.0,3.0,NaN,-1.0,NaN,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,NaN,2.0,1.0,NaN,-1.0,NaN,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,NaN,NaN,NaN,NaN,spala101,Al Spalding,braia102,Asa Brainard,wrigg101,George Wright,6.0,barnr102,Ross

In [9]:
game_log.tail()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
171902,20161002,0,Sun,MIL,NL,162,COL,NL,162,6,4,60.0,D,NaN,NaN,NaN,DEN02,27762.0,203.0,0200000202,1100100010,39.0,10.0,4.0,1.0,2.0,6.0,0.0,0.0,1.0,4.0,0.0,12.0,2.0,1.0,0.0,0.0,8.0,7.0,4.0,4.0,1.0,0.0,30.0,12.0,1.0,0.0,0.0,0.0,41.0,13.0,4.0,0.0,1.0,4.0,1.0,0.0,1.0,3.0,0.0,11.0,0.0,1.0,0.0,0.0,12.0,5.0,6.0,6.0,0.0,0.0,30.0,13.0,0.0,0.0,0.0,0.0,barrs901,Scott Barry,woodt901,Tom Woodring,randt901,Tony Randazzo,ortir901,Roberto Ortiz,NaN,NaN,NaN,NaN,counc001,Craig Counsell,weisw001,Walt Weiss,thort001,Tyler Thornburg,rusic001,Chris Rusin,knebc001,Corey Knebel,susaa001,Andrew Susac,cravt001,Tyler Cravy,marqg001,German Marquez,villj001,Jonathan Villar,5.0,genns001,Scooter Gennett,4.0,cartc002,Chris Carter,3.0,santd002,Domingo Santana,9.0,pereh001,Hernan Perez,8.0,arcio002,Orlando Arcia,6.0,susaa001,Andrew Susac,2.0,elmoj001,Jake Elmore,7.0,cravt001,Tyler Cravy,1.0,blacc001,Charlie Blackmon,8.0,dahld001,David Dahl,7.0,arenn001,Nolan Arenado,5.0,gonzc001,Carlos Gonzalez,9.0,murpt002,Tom Murphy,2.0,pattj005,Jordan Patterson,3.0,valap001,Pat Valaika,4.0,adamc001,Cristhian Adames,6.0,marqg001,German Marquez,1.0,NaN,Y
171903,20161002,0,Sun,NYN,NL,162,PHI,NL,162,2,5,51.0,D,NaN,NaN,NaN,PHI13,36935.0,159.0,000001100,00100031x,33.0,8.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,9.0,1.0,1.0,1.0,0.0,6.0,6.0,3.0,3.0,0.0,0.0,24.0,12.0,3.0,1.0,2.0,0.0,33.0,10.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,2.0,0.0,3.0,0.0,0.0,2.0,0.0,7.0,5.0,2.0,2.0,0.0,0.0,27.0,7.0,0.0,0.0,1.0,0.0,barkl901,Lance Barksdale,herna901,Angel Hernandez,barrt901,Ted Barrett,littw901,Will Little,NaN,NaN,NaN,NaN,collt801,Terr

In [10]:
# import park_codes.csv
park_codes = pd.read_csv("park_codes.csv", low_memory=False)
park_codes.shape

(252, 9)

In [11]:
park_codes.head()

,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,04/30/1884,05/31/1884,UA,NaN
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,NaN,AL,NaN
3,ARL01,Arlington Stadium,NaN,Arlington,TX,04/21/1972,10/03/1993,AL,NaN
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,04/11/1994,NaN,AL,NaN


In [12]:
park_codes.tail()

,park_id,name,aka,city,state,start,end,league,notes
247,WIL02,BB&T Ballpark at Bowman Field,NaN,Wiliamsport,PA,08/20/2017,08/20/2017,NL,PIT
248,WNY01,West New York Field Club Grounds,NaN,West New York,NJ,09/11/1898,09/17/1899,NL,"BRO:9/18&10/2/1898; NY1:9/11/98, 6/4&7/16&8/13&9/17/99"
249,WOR01,Agricultural County Fair Grounds I,NaN,Worcester,MA,05/01/1880,09/29/1882,NL,NaN
250,WOR02,Agricultural County Fair Grounds II,NaN,Worcester,MA,08/17/1887,08/17/1887,NL,1 BSN game
251,WOR03,Worcester Driving Park Grounds,NaN,Worcester,MA,10/30/1874,10/30/1874,NaN,1 BS1 game


In [13]:
# import person_codes.csv
person_codes = pd.read_csv("person_codes.csv", low_memory=False)
person_codes.shape

(20494, 7)

In [14]:
person_codes.head()

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN
2,aarot101,Aaron,Tommie,04/10/1962,NaN,04/06/1979,NaN
3,aased001,Aase,Don,07/26/1977,NaN,NaN,NaN
4,abada001,Abad,Andy,09/10/2001,NaN,NaN,NaN


In [15]:
person_codes.tail()

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
20489,zuvep001,Zuvella,Paul,09/04/1982,NaN,04/02/1996,NaN
20490,zuveg101,Zuverink,George,04/21/1951,NaN,NaN,NaN
20491,zwild101,Zwilling,Dutch,08/14/1910,NaN,04/15/1941,NaN
20492,zycht001,Zych,Tony,09/04/2015,NaN,NaN,NaN
20493,thoma102,Thompson,NaN,NaN,NaN,NaN,NaN


In [16]:
# import team_codes.csv
team_codes = pd.read_csv("team_codes.csv", low_memory=False)
team_codes.shape

(150, 8)

In [17]:
team_codes.head()

,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
2,BFN,NL,1879,1885,Buffalo,Bisons,BFN,1
3,BFP,PL,1890,1890,Buffalo,Bisons,BFP,1
4,BL1,NaN,1872,1874,Baltimore,Canaries,BL1,1


In [18]:
team_codes.tail()

,team_id,league,start,end,city,nickname,franch_id,seq
145,WS8,NL,1886,1889,Washington,Senators,WS8,1
146,WS9,AA,1891,1891,Washington,Senators,WS9,1
147,WSN,NL,1892,1899,Washington,Senators,WS9,2
148,WSU,UA,1884,1884,Washington,Nationals,WSU,1
149,MIA,NL,2012,0,Miami,Marlins,FLO,2


With 161 columns and 171907 rows, game_log is a behemoth of a table and has a fantastic amount of data inside. Each row contains information about a particular game, with dates ranging from 1871 to 2016. It is well organized and the accompanying text file will be very helpful for interpreting each column. game_log lists the defensive positions by number rather than writing it out. The conventional mapping of these numbers is as follows:

1. pitcher
2. catcher
3. first baseman
4. second baseman
5. third baseman
6. shortstop
7. left fielder
8. center fielder
9. right fielder

The remaining 3 files are helpers that intersect with game_log. 
- park_codes contains 9 columns of information about 252 ball parks, and intersects the main log file on the park_id column.
- person_codes contains 20494 rows of people (players, managers, coaches, and umpires), with columns for their id, name, and debut date(s). This table gives each player a unique id, and these ids occur in many places throughout game_log adjacent to columns mentioning a person.
- team_codes contains 150 rows of teams, with 8 columns of information about each. It intersects with game_log on the team_id column. It also contains a franch_id column, but this does not appear to be referenced anywhere else.

One last thing to note is the "league" identifier. There is actually a league column in game_log, park_codes, AND team_codes. Note that all but the National and American Leagues are now defunct. The leagues included are:
- UA - Union Association
- NL - National League
- PL - Players' League
- AA - American Association
- AL - American League
- FL - Federal League

In [19]:
# create game_log table in mlb.db database
game_log.to_sql('game_log', sqlite3.connect('mlb.db'), if_exists='replace', index=False)

In [20]:
# add game_id column to game_log table
c = """
ALTER TABLE game_log
ADD COLUMN game_id TEXT;
"""
run_command(c)

In [21]:
# add game_id values by concatenating the home team name ID, date, 
# and number_of_game, as this format is used elsewhere by Retrosheet
d = """
UPDATE game_log
SET game_id= h_name || date || number_of_game;
"""
run_command(d)

In [22]:
# create park_codes table in mlb.db database
park_codes.to_sql('park_codes', sqlite3.connect('mlb.db'), if_exists='replace', index=False)

In [23]:
# create person_codes table in mlb.db database
person_codes.to_sql('person_codes', sqlite3.connect('mlb.db'), if_exists='replace', index=False)

In [24]:
# create team_codes table in mlb.db database
team_codes.to_sql('team_codes', sqlite3.connect('mlb.db'), if_exists='replace', index=False)

## Data normalization
Now that we've imported our data into sqlite, let's work on normalizing it. 

### game_log
In the game_log table, there are a number of columns we can eliminate, as they're redundant across our tables, A prime target is the name columns of people, as these are all represented in the person_codes table and we have a link between the tables with person IDs. These means we should be able to outright delete:
* hp_umpire_name
* 1b_umpire_id
* 2b_umpire_name
* 3b_umpire_name
* lf_umpire_name
* rf_umpire_name
* v_manager_name
* h_manager_name
* winning_pitcher_name
* losing_pitcher_name
* saving_pitcher_name
* winning_rbi_batter_id_name
* v_starting_pitcher_name
* h_starting_pitcher_name
* v_player_1_name ... v_player_9_name
* h_player_1_name ... h_player_9_name

That by itself is 32 columns we can eliminate! That'll feel good.
Additionally, we can potentially relocate all of the player_def_pos and off_pos identifiers over to our player table. The one complication with this is that if those features aren't static over a player's career then it probably still makes sense to leave those as game-specific parameters.

### park_codes
In the park_codes table, the start, end, and league columns are all redundant, as we can derive all of this information from elsewhere in our database.

### person_codes
For the person table, we can eliminate the debut dates, as all of these can be derived from the game_log table. We've also decided to ditch the coaches altogether, since they're not actually mentioned anywhere else.

## Schema
We'll develop our database into a schema with the following interconnected tables (I'd post a picture, but DQ won't let me. Maybe later):

* team_appearance
    * PK - team_id --> team
    * FK - game_id --> game
    * FK - league_id --> league
* league
    * league_id --> team_appearance, team
* team
    * PK - team_id <-> team_appearance
    * FK - league_id --> league
* person_appearance
    * PK - appearance_id
    * FK - person_id --> person
    * FK - team_id --> team
    * FK - game_id --> game
    * FK - appearance_type_id --> appearance_type
* person
    * PK - person_id  --> person_appearance
* appearance_type
    * PK - appearance_type_id
* game
    * PK - game_id --> team_appearance, person_appearance
    * FK - park_id --> park
* park
    * PK - park_id --> game
## Creating the Tables

In [25]:
# create person table if not exists and define columns
create_person_table = """
CREATE TABLE IF NOT EXISTS person(
    person_id BLOB PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
);
"""
run_command(create_person_table)

In [26]:
# insert data from person_codes into person table if not already there
insert_person_data = """
INSERT OR IGNORE INTO person
SELECT id, first, last FROM person_codes;
"""

run_command(insert_person_data)

In [27]:
run_query('SELECT * FROM person LIMIT 5;')

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


In [28]:
# create park table if not exists and define columns
create_park_table = """
CREATE TABLE IF NOT EXISTS park(
    park_id BLOB PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT
);
"""
run_command(create_park_table)

In [29]:
# insert data from park_codes into park table if not already there
insert_park_data = """
INSERT OR IGNORE INTO park
SELECT park_id, name, aka, city, state, notes FROM park_codes;
"""

run_command(insert_park_data)

In [30]:
run_query('SELECT * FROM park LIMIT 5;')

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None


In [31]:
# create league table if not exists and define columns
create_league_table = """
CREATE TABLE IF NOT EXISTS league(
    league_id TEXT PRIMARY KEY,
    name TEXT
);
"""
run_command(create_league_table)

In [32]:
# insert data into league table if not already there
insert_league_data = """
INSERT OR IGNORE INTO league VALUES 
    ("AA", "American Association"),
    ("AL", "American League"),
    ("FL", "Federal League"),
    ("NL", "National League"),
    ("PL", "Player's League"),
    ("UA", "Union Association"),
    ("None", "None");
"""

run_command(insert_league_data)

In [33]:
run_query('SELECT * FROM league LIMIT 10;')

,league_id,name
0,AA,American Association
1,AL,American League
2,FL,Federal League
3,NL,National League
4,PL,Player's League
5,UA,Union Association
6,None,None


In [34]:
# import appearance_type.csv
appearance_type = pd.read_csv("appearance_type.csv", low_memory=False)
appearance_type.head()

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense


In [35]:
# create appearance table in mlb.db database
# unfortunately this stupid process is necessary because we can't create
# a primary key with the pandas to_sql method
appearance_type.to_sql('appearance', sqlite3.connect('mlb.db'), 
                  if_exists='replace', index=False)

In [36]:
# create appearance_type table if not exists and define columns
create_appearance_type = """
CREATE TABLE IF NOT EXISTS appearance_type(
    appearance_type_id BLOB PRIMARY KEY,
    name TEXT,
    category TEXT
);
"""
run_command(create_appearance_type)

In [37]:
# insert data from appearance into appearance_type table if not already there
insert_appearance_data = """
INSERT OR IGNORE INTO appearance_type
SELECT * FROM appearance;
"""

run_command(insert_appearance_data)

In [38]:
run_query('SELECT * FROM appearance_type LIMIT 5;')

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense


In [39]:
# create team table if not exists and define columns
create_team = """
CREATE TABLE IF NOT EXISTS team(
    team_id BLOB PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id BLOB,
    FOREIGN KEY(league_id) REFERENCES league(league_id)
);
"""
run_command(create_team)

In [40]:
# insert data from team_codes into team table if not already there
insert_team_data = """
INSERT OR IGNORE INTO team
SELECT team_id, league, city, nickname, franch_id FROM team_codes;
"""

run_command(insert_team_data)

In [41]:
run_query('SELECT * FROM team LIMIT 5;')

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


In [42]:
# create game table if not exists and define columns
create_game = """
CREATE TABLE IF NOT EXISTS game(
    game_id BLOB PRIMARY KEY,
    date BLOB,
    number_of_game INTEGER,
    park_id BLOB,
    length_outs INTEGER,
    day BLOB,
    completion BLOB,
    forfeit BLOB,
    protest BLOB,
    attendance NUMERIC,
    length_minutes NUMERIC,
    additional_info BLOB,
    acquisition_info BLOB,
    FOREIGN KEY(park_id) REFERENCES park(park_id)
);
"""
run_command(create_game)

In [43]:
# insert data from game_log into game table if not already there
insert_game_data = """
INSERT OR IGNORE INTO game
SELECT 
    game_id, date, number_of_game, park_id, length_outs, day_night, 
    completion, forefeit, protest, attendance, length_minutes, 
    additional_info, acquisition_info 
FROM game_log;
"""

run_command(insert_game_data)

In [44]:
run_query('SELECT * FROM game LIMIT 5;')

,game_id,date,number_of_game,park_id,length_outs,day,completion,forfeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54,D,None,None,None,200,120,None,Y
1,WS3187105050,18710505,0,WAS01,54,D,None,None,None,5000,145,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54,D,None,None,None,1000,140,None,Y
3,CH1187105080,18710508,0,CHI01,54,D,None,None,None,5000,150,None,Y
4,TRO187105090,18710509,0,TRO01,54,D,None,None,None,3250,145,HTBF,Y


In [45]:
run_query('SELECT * FROM game_log LIMIT 5')

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info,game_id
0,18710504,0,Thu,CL1,None,1,FW1,None,1,0,2,54.0,D,None,None,None,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,None,6.0,1.0,None,-1.0,None,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,None,0.0,0.0,None,-1.0,None,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,None,None,None,None,None,None,None,None,None,None,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,None,None,None,None,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,None,Y,FW1187105040
1,18710505,0,Fri,BS1,None,1,WS3,None,1,20,18,54.0,D,None,None,None,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,None,5.0,3.0,None,-1.0,None,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,None,2.0,1.0,None,-1.0,None,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,None,None,None,None,None,None,None,None,None,None,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,None,None,None,None,spala101,Al

In [46]:
# create team_appearance table if not exists and define columns
run_command("""
    CREATE TABLE IF NOT EXISTS team_appearance(
        team_id TEXT,
        game_id TEXT,
        home INTEGER,
        league_id TEXT,
        score INTEGER,
        line_score TEXT,
        at_bats REAL,
        hits REAL,
        doubles REAL,
        triples REAL,
        homeruns REAL,
        rbi REAL,
        sacrifice_hits REAL,
        sacrifice_flies REAL,
        hit_by_pitch REAL,
        walks REAL,
        intentional_walks REAL,
        strikeouts REAL,
        stolen_bases REAL,
        caught_stealing REAL,
        grounded_into_double REAL,
        first_catcher_interference REAL,
        left_on_base REAL,
        pitchers_used REAL,
        individual_earned_runs REAL,
        team_earned_runs REAL,
        wild_pitches REAL,
        balks REAL,
        putouts REAL,
        assists REAL,
        errors REAL,
        passed_balls REAL,
        double_plays REAL,
        triple_plays REAL,
        PRIMARY KEY(team_id, game_id),
        FOREIGN KEY(game_id) REFERENCES game(game_id),
        FOREIGN KEY(league_id) REFERENCES league(league_id)
    );
    """)

In [47]:
# insert data from game_log into game table if not already there
run_command("""
    INSERT OR IGNORE INTO team_appearance
        SELECT
            h_name,
            game_id,
            1 AS home,
            h_league,
            h_score,
            h_line_score,
            h_at_bats,
            h_hits,
            h_doubles,
            h_triples,
            h_homeruns,
            h_rbi,
            h_sacrifice_hits,
            h_sacrifice_flies,
            h_hit_by_pitch,
            h_walks,
            h_intentional_walks,
            h_strikeouts,
            h_stolen_bases,
            h_caught_stealing,
            h_grounded_into_double,
            h_first_catcher_interference,
            h_left_on_base,
            h_pitchers_used,
            h_individual_earned_runs,
            h_team_earned_runs,
            h_wild_pitches,
            h_balks,
            h_putouts,
            h_assists,
            h_errors,
            h_passed_balls,
            h_double_plays,
            h_triple_plays
        FROM game_log

    UNION

        SELECT    
            v_name,
            game_id,
            0 AS home,
            v_league,
            v_score,
            v_line_score,
            v_at_bats,
            v_hits,
            v_doubles,
            v_triples,
            v_homeruns,
            v_rbi,
            v_sacrifice_hits,
            v_sacrifice_flies,
            v_hit_by_pitch,
            v_walks,
            v_intentional_walks,
            v_strikeouts,
            v_stolen_bases,
            v_caught_stealing,
            v_grounded_into_double,
            v_first_catcher_interference,
            v_left_on_base,
            v_pitchers_used,
            v_individual_earned_runs,
            v_team_earned_runs,
            v_wild_pitches,
            v_balks,
            v_putouts,
            v_assists,
            v_errors,
            v_passed_balls,
            v_double_plays,
            v_triple_plays
        from game_log;
    """)

In [48]:
run_query('''
SELECT * FROM team_appearance 
ORDER BY score DESC
LIMIT 150
''')

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,PH1,TRO187106280,0,None,49,243798358,73.0,42.0,8.0,5.0,2.0,31.0,0.0,0.0,0.0,7.0,NaN,0.0,4.0,NaN,-1.0,NaN,4.0,1.0,23.0,23.0,2.0,0.0,27.0,11.0,10.0,4.0,0.0,0.0
1,BL1,BR2187210050,0,None,39,(11)13621492,77.0,42.0,5.0,2.0,0.0,30.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,2.0,0.0,NaN,11.0,1.0,2.0,0.0,1.0,0.0,27.0,9.0,13.0,7.0,0.0,0.0
2,NY2,NY2187406180,1,None,38,815358521,72.0,34.0,3.0,5.0,4.0,23.0,0.0,0.0,0.0,1.0,NaN,0.0,4.0,0.0,0.0,NaN,8.0,1.0,0.0,0.0,0.0,0.0,27.0,6.0,4.0,0.0,0.0,0.0
3,TRO,TRO187107030,1,None,37,112239946,73.0,36.0,5.0,4.0,1.0,28.0,0.0,0.0,0.0,3.0,NaN,1.0,2.0,NaN,-1.0,NaN,12.0,1.0,13.0,13.0,1.0,0.0,27.0,12.0,8.0,1.0,1.0,0.0
4,CHN,CHN189706290,1,NL,36,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MID,MID187206220,1,None,36,335554704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,BL1,BL1187306300,1,None,35,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,CHN,CHN188207240,1,NL,35,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,PH1,PH1187205010,1,None,34,024358606,70.0,34.0,5.0,4.0,0.0,28.0,0.0,0.0,0.0,2.0,NaN,4.0,1.0,0.0,0.0,NaN,11.0,1.0,4.0,0.0,0.0,0.0,27.0,12.0,11.0,0.0,0.0,0.0
9,TRO,TRO187106280,1,None,33,123(10)44243,64.0,32.0,6.0,5.0,1.0,23.0,0.0,0.0,0.0,6.0,NaN,2.0,11.0,NaN,-1.0,NaN,10.0,1.0,31.0,31.0,5.0,0.0,27.0,12.0,10.0,6.0,1.0,0.0


In [49]:
# create person_appearance table if not exists and define columns
run_command("""
CREATE TABLE IF NOT EXISTS person_appearance(
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id TEXT,
    FOREIGN KEY(person_id) REFERENCES person(person_id),
    FOREIGN KEY(team_id) REFERENCES team(team_id),
    FOREIGN KEY(game_id) REFERENCES game(game_id),
    FOREIGN KEY(appearance_type_id) REFERENCES appearance_type(appearance_type_id)
    );
    """)

In [51]:
# insert data into person_appearance if not already there
insert_person_appearance_data = """
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
)

    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE [1b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN v_name
            ELSE h_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "AWB"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "AWB"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL
"""
run_command(insert_person_appearance_data)

In [57]:
template = """
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""

#run_command(c1)
#run_command(c2)

for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        # run commmand is a helper function which runs
        # a query against our database.
        run_command(template.format(**query_vars))

In [65]:
run_query('''
SELECT 
    DISTINCT appearance_type_id
FROM person_appearance 
ORDER BY appearance_type_id
LIMIT 30
''')

,appearance_type_id
0,ALP
1,ASP
2,AWB
3,AWP
4,D1
5,D10
6,D2
7,D3
8,D4
9,D5


In [66]:
run_command('''DROP TABLE IF EXISTS game_log;''')
run_command('''DROP TABLE IF EXISTS park_codes;''')
run_command('''DROP TABLE IF EXISTS team_codes;''')
run_command('''DROP TABLE IF EXISTS person_codes;''')